<a href="https://colab.research.google.com/github/Michel-p16/PDS-Project/blob/Michel/DataGeneration_PDS_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files

# Datei hochladen
uploaded = files.upload()

# Zeigt die hochgeladene Datei an
for file_name in uploaded.keys():
    print(f"Datei {file_name} wurde hochgeladen.")

Saving questionnaire1.json to questionnaire1.json
Saving questionnaire2.json to questionnaire2.json
Saving questionnaire3.json to questionnaire3.json
Saving questionnaire4.json to questionnaire4.json
Saving questionnaire5.json to questionnaire5.json
Datei questionnaire1.json wurde hochgeladen.
Datei questionnaire2.json wurde hochgeladen.
Datei questionnaire3.json wurde hochgeladen.
Datei questionnaire4.json wurde hochgeladen.
Datei questionnaire5.json wurde hochgeladen.


In [5]:
import json

file_names = [
    "questionnaire1.json",
    "questionnaire2.json",
    "questionnaire3.json",
    "questionnaire4.json",
    "questionnaire5.json"
]

data_list = []

# Jede Datei einzeln lesen
for file_name in file_names:
    with open(file_name, 'r') as f:
        data = json.load(f)
        data_list.append(data)

# Daten für jede Datei anzeigen
for i, data in enumerate(data_list):
    print(f"Datei {file_names[i]} enthält {len(data)} Fragen.")

Datei questionnaire1.json enthält 5 Fragen.
Datei questionnaire2.json enthält 4 Fragen.
Datei questionnaire3.json enthält 4 Fragen.
Datei questionnaire4.json enthält 5 Fragen.
Datei questionnaire5.json enthält 7 Fragen.


In [6]:
# Beispiel: Alle Fragen aus allen Dateien ausgeben
for file_data in data_list:
    for question in file_data:
        print(f"Frage: {question['question']}, Typ: {question['type']}")

Frage: Data processing consent, Typ: SINGLE_SELECT
Frage: Customer group, Typ: SINGLE_SELECT
Frage: Products interested in, Typ: MULTI_SELECT
Frage: What kind of follow up is planned, Typ: MULTI_SELECT
Frage: Who to copy in follow up, Typ: MULTI_SELECT
Frage: Would you like to receive marketing information from via e-mail?, Typ: SINGLE_SELECT
Frage: What industry are you operating in?, Typ: SINGLE_SELECT
Frage: What products are you interested in?, Typ: MULTI_SELECT
Frage: Notes, Typ: TEXT
Frage: What type of company is it?, Typ: SINGLE_SELECT
Frage: What is the size of your company?, Typ: SINGLE_SELECT
Frage: When do you wish to receive a follow-up?, Typ: DATE
Frage: Any additional notes?, Typ: TEXT
Frage: Which language is wanted for communication? , Typ: SINGLE_SELECT
Frage: What is the type of contact?, Typ: MULTI_SELECT
Frage: What is the contact person interested in?, Typ: MULTI_SELECT
Frage: What phone number can we use for contact?, Typ: NUMBER
Frage: When does the contact pers

In [7]:
combined_questions = [q for file_data in data_list for q in file_data]
print(f"Insgesamt {len(combined_questions)} Fragen aus allen Dateien.")

Insgesamt 25 Fragen aus allen Dateien.


In [8]:
import google.generativeai as genai
import time
from google.colab import userdata

# API-Schlüssel abrufen, der in Colab Secrets gespeichert wurde
key = userdata.get('GOOGLE_API_KEY')

# API-Schlüssel konfigurieren, um die Gemini API zu nutzen
if key:
    genai.configure(api_key=key)
else:
    raise ValueError("API-Schlüssel konnte nicht abgerufen werden. Bitte sicherstellen, dass der Schlüssel in Colab Secrets gespeichert wurde.")

# Modell laden
model = genai.GenerativeModel("gemini-1.5-flash")

# Funktion, um die Frage mit der Gemini API zu reformulieren
def reformulate_question(question_text):
    try:
        prompt = f"Rewrite the following text as a proper question to a customer: '{question_text}. Keep it short and natural'"
        response = model.generate_content(
            prompt,
            generation_config={
                "temperature": 0.7,
                "max_output_tokens": 50
            }
        )
        return response.text.strip() if response and response.text else question_text
    except Exception as e:
        print(f"Fehler bei der Frageumformulierung: {e}")
        return question_text

# Fragen aus `combined_questions` reformulieren
for question in combined_questions:
    original_question = question["question"]
    question["question"] = reformulate_question(original_question)
    print(f"Original: {original_question}\nReformuliert: {question['question']}\n")
    time.sleep(5)  # Verzögerung von 5 Sekunden zwischen den Anfragen

# Ergebnisse speichern
output_file = 'reformulated_combined_questions.json'
with open(output_file, 'w') as f:
    json.dump(combined_questions, f, indent=4)

print(f"Reformulierte Fragen wurden in '{output_file}' gespeichert.")

# Datei herunterladen
files.download(output_file)

Original: Data processing consent
Reformuliert: May we process your data?

Original: Customer group
Reformuliert: What's your customer group?

Original: Products interested in
Reformuliert: Which products are you interested in?

Original: What kind of follow up is planned
Reformuliert: What's the follow-up plan?

Original: Who to copy in follow up
Reformuliert: Who should I CC on the follow-up?

Original: Would you like to receive marketing information from via e-mail?
Reformuliert: Want marketing emails?

Original: What industry are you operating in?
Reformuliert: What industry are you in?

Original: What products are you interested in?
Reformuliert: What are you interested in?

Original: Notes
Reformuliert: Could you please leave a short note?

Original: What type of company is it?
Reformuliert: What kind of company is this?

Original: What is the size of your company?
Reformuliert: How big is your company?

Original: When do you wish to receive a follow-up?
Reformuliert: When would 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# List of prompts

prompt_templates = {
    "SINGLE_SELECT": [
        "Imagine you are a user. Respond to the question '{question_text}' by choosing the option '{option}'. Provide only the response, and do not include any explanations or examples. Be concise and realistic. Use about 7 words",
        "Based on the option '{option}', how would a typical user answer the question: '{question_text}'?. Provide only the response, and do not include any explanations or examples. Use around 20 words for the answer",
        "As a user, give a clear and concise answer to the question: '{question_text}' while including the option '{option}'. Provide only the response, and do not include any explanations or examples. use about 5 words",
        "Respond conversationally to the question: '{question_text}', incorporating the choice '{option}'. Provide only the response, and do not include any explanations or examples.",
        "What would a user realistically say to this question: '{question_text}', if they chose '{option}'? Provide only the response, and do not include any explanations or examples. Use around 20 words for the answer"
    ],

    "MULTI_SELECT": [
        "If a user selects the options '{options}', how would they naturally respond to the question: '{question_text}'? Only provide the answer and nothing else. Use about 20 Words",
        "Imagine a user answering the question: '{question_text}' based on the selected options '{options}'. Keep it realistic and concise. Only provide the answer and nothing else. Use about 10-15 words",
        "What would a typical user say to the question: '{question_text}', considering their choices '{options}'? Make it sound conversational. Use around 40 words",
        "Generate a realistic, thoughtful response to '{question_text}' while using the choices '{options}' naturally. Only provide the answer and nothing else",
        "Respond as a user might: '{question_text}', with selected options '{options}'. Keep it engaging but realistic. Only provide the answer and nothing else"
    ],

    "TEXT": [
        "Write a brief and realistic response to the question: {question_text}. Only provide the answer and nothing else",
        "What might a user naturally say in response to: {question_text}? Keep it informal and user-focused. Only provide the answer and nothing else",
        "Provide a thoughtful answer to '{question_text}', as a user might say in a real-world situation. Only provide the answer and nothing else",
        "Answer the question '{question_text}' in a way that sounds natural and engaging, as if you were the user. Only provide the answer and nothing else",
        "Generate a short, conversational response to the question: '{question_text}'. Only provide the answer and nothing else"
    ],

    "DATE": [
        "Provide a realistic date for the question: '{question_text}'. Keep it plausible. Only provide the answer and nothing else",
        "What date might a user reply with for '{question_text}'? Be concise and realistic. Only provide the answer and nothing else",
        "Respond to '{question_text}' with a natural and plausible date in far future. Only provide the answer and nothing else",
        "Imagine a user answering the question '{question_text}'. Provide a realistic date in near future they might give. Only provide the answer and nothing else",
        "Generate a user-like response with a date in about 4 month to '{question_text}'. Only provide the answer and nothing else"
    ],

    "NUMBER": [
        "Provide a realistic number in response to the question: '{question_text}'. Only provide the answer and nothing else containing a single number",
        "What would a user answer to the question '{question_text}' with a number? Be realistic and concise. Only provide the answer and nothing else",
        "Answer the question '{question_text}' as a user, using a plausible number. Only provide the answer and nothing else",
        "Generate a thoughtful and natural response to '{question_text}', including a realistic number. Only provide the answer and nothing else",
        "What might a user realistically say to '{question_text}' with a number? Only provide the answer and nothing else"
    ]

}


In [12]:
import random
import json
import time
from datetime import datetime
from google.colab import files

def generate_response_gemini(prompt):
    try:
        response = model.generate_content(
            prompt,
            generation_config={
                "temperature": 0.7,
                "max_output_tokens": 150
            }
        )
        return response.text.strip() if response and response.text else None
    except Exception as e:
        print(f"Fehler bei der Antwortgenerierung: {e}")
        return None

# Reformulierte Fragen laden
input_file = 'reformulated_combined_questions.json'
with open(input_file, 'r') as f:
    combined_questions = json.load(f)

# Anzahl der Antworten pro Fragetyp
target_counts = {
    "SINGLE_SELECT": 700,
    "MULTI_SELECT": 500,
    "TEXT": 100,
    "DATE": 100,
    "NUMBER": 100
}

# Zähler für generierte Antworten pro Fragetyp
generated_counts = {key: 0 for key in target_counts}

# Simulierte Antworten erstellen
generated_data = []

for question in combined_questions:
    if all(generated_counts[qt] >= target_counts[qt] for qt in target_counts):
        break  # Abbrechen, wenn die Zielanzahl erreicht ist

    question_id = question["id"]
    question_text = question["question"]
    question_type = question["type"]
    options = question.get("options", [])

    # Skip questions if their type has already reached the target count
    if generated_counts[question_type] >= target_counts[question_type]:
        continue

    answers = []

    if question_type == "SINGLE_SELECT":
        for option in options:
            if generated_counts["SINGLE_SELECT"] >= target_counts["SINGLE_SELECT"]:
                break

            if random.random() < 0.2:  # 20% Wahrscheinlichkeit für alternative Antwort
                response = random.choice(["I don't know", "I don't care", ""])
                answers.append({
                    "text": response,
                    "label": "No Preference",
                    "timestamp": datetime.now().isoformat()
                })
            else:
                option_text = option["option"]
                prompt = random.choice(prompt_templates["SINGLE_SELECT"]).format(
                    question_text=question_text,
                    option=option_text
                )
                response = generate_response_gemini(prompt)
                time.sleep(5)  # Verzögerung
                answers.append({
                    "text": response if response else "No Response",
                    "label": option_text,
                    "timestamp": datetime.now().isoformat()
                })
            generated_counts["SINGLE_SELECT"] += 1

    elif question_type == "MULTI_SELECT":
        for _ in range(5):
            if generated_counts["MULTI_SELECT"] >= target_counts["MULTI_SELECT"]:
                break

            if random.random() < 0.2:
                response = random.choice(["I don't know", "I don't care", ""])
                answers.append({
                    "text": response,
                    "label": "No Preference",
                    "timestamp": datetime.now().isoformat()
                })
            else:
                selected_options = random.sample(options, k=random.randint(1, len(options)))
                selected_texts = [opt["option"] for opt in selected_options]
                prompt = random.choice(prompt_templates["MULTI_SELECT"]).format(
                    question_text=question_text,
                    options=", ".join(selected_texts)
                )
                response = generate_response_gemini(prompt)
                time.sleep(5)
                answers.append({
                    "text": response if response else "No Response",
                    "label": selected_texts,
                    "timestamp": datetime.now().isoformat()
                })
            generated_counts["MULTI_SELECT"] += 1

    elif question_type in ["TEXT", "DATE", "NUMBER"]:
        for _ in range(5):
            if generated_counts[question_type] >= target_counts[question_type]:
                break

            if random.random() < 0.2:
                response = random.choice(["I don't know", "I don't care", ""])
                answers.append({
                    "text": response,
                    "label": "No Preference",
                    "timestamp": datetime.now().isoformat()
                })
            else:
                prompt = random.choice(prompt_templates[question_type]).format(
                    question_text=question_text
                )
                response = generate_response_gemini(prompt)
                time.sleep(5)
                answers.append({
                    "text": response if response else "No Response",
                    "label": question_type.capitalize(),
                    "timestamp": datetime.now().isoformat()
                })
            generated_counts[question_type] += 1

    # Antwort zur Liste hinzufügen
    generated_data.append({
        "question_id": question_id,
        "question": question_text,
        "type": question_type,
        "answers": answers
    })

# Ergebnisse in einer JSON-Datei speichern
output_file = 'generated_responses.json'
with open(output_file, 'w') as f:
    json.dump(generated_data, f, indent=4)

print(f"Simulierte Antworten wurden in '{output_file}' gespeichert.")

Simulierte Antworten wurden in 'generated_responses_natural_language.json' gespeichert.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# Datei herunterladen
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import pandas as pd

# JSON-Datei laden
with open(output_file, 'r') as f:
    data = json.load(f)

# Daten in ein DataFrame umwandeln
rows = []
for entry in data:
    question_id = entry["question_id"]
    question = entry["question"]
    question_type = entry["type"]
    for answer in entry["answers"]:
        rows.append({
            "question_id": question_id,
            "question": question,
            "type": question_type,
            "answer_text": answer["text"],
            "answer_label": answer["label"]
        })

# DataFrame erstellen
df = pd.DataFrame(rows)

# DataFrame anzeigen
print(df.head())

# DataFrame speichern (optional)
output_file = 'questions_and_answers.csv'
df.to_csv(output_file, index=False)
print(f"DataFrame wurde in '{output_file}' gespeichert.")


                            question_id                     question  \
0  aa2d8cdd-0758-4035-b0b6-ca18e2f380d8    May we process your data?   
1  aa2d8cdd-0758-4035-b0b6-ca18e2f380d8    May we process your data?   
2  12e1ed1d-edaa-4e93-8645-de3850e998f9  What's your customer group?   
3  12e1ed1d-edaa-4e93-8645-de3850e998f9  What's your customer group?   
4  12e1ed1d-edaa-4e93-8645-de3850e998f9  What's your customer group?   

            type                                        answer_text  \
0  SINGLE_SELECT                                       I don't care   
1  SINGLE_SELECT   No, I do not consent to my data being processed.   
2  SINGLE_SELECT                          End users and businesses.   
3  SINGLE_SELECT  We primarily serve wholesalers and distributor...   
4  SINGLE_SELECT             Consultants, planners, and architects.   

                     answer_label  
0                   No Preference  
1                              No  
2                        End Use